# Contar doses de vacinas da COVID-19

In [81]:
import pandas as pd
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


### Dica: Restrinja seus testes aos estados do AC, AP e RR para diminuir o tamanho da sua Query.

In [ ]:
## Defina o id do seu projeto no bigquery!!!!!
project_id = 'enap-331414' # Defina o id do seu projeto no bigquery!!!!!
## Defina o id do seu projeto no bigquery!!!!!

df_pibpercapita = pd.io.gbq.read_gbq('''
SELECT 
    sigla_uf
    ,vacina
    ,count(*) as qt_total
    ,sum(case when regexp_contains(dose,'1ª Dose|^Dose$|Dose Inicial') then 1 else 0 end) as qt_D1
    ,sum(case when regexp_contains(dose,'2ª Dose') then 1 else 0 end) as qt_D2
    ,sum(case when regexp_contains(dose,'Reforço') then 1 else 0 end) as qt_Reforco
    ,sum(case when regexp_contains(dose,'Doses Adicional|3ª Dose') then 1 else 0 end) as qt_Adicional
    ,sum(case when regexp_contains(dose,'nica') then 1 else 0 end) as qt_Unica
FROM `basedosdados.br_ms_vacinacao_covid19.microdados_vacinacao`
where sigla_uf in ('AC','AP','RR')
group by sigla_uf, vacina;
''', project_id=project_id)

df_pibpercapita.head()

### 6) Enviar o dataframe de resultado ao BigQuery. 
#### Decida se lhe é conveniente reutilizar a tabela e o data source no Data Studio, ou se deseja criar uma nova tabela e um novo data source.
#### Dica 1: Se esta operação apenas adicionar colunas à tabela, sem remover outras, pode reutilizar a tabela e o data source.
#### Dica 2: Se reutilizar a tabela, atualize os campos do seu data source.


In [ ]:
dfm.to_gbq("enapdatasets.pibpercapita",
  project_id=project_id,
  chunksize=40000,
  if_exists='replace',
  )

5it [00:39,  7.86s/it]
